In [2]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
#from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import time
from lxml import etree
from alive_progress import alive_bar #（pip install alive_progress）进度条
import re
import csv
#from openpyxl import Workbook
# from tqdm.notebook import trange
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import traceback
import re 
from multiprocessing.dummy import Pool
import itertools


# 数据爬取

In [3]:
def get_page(url):
    try:
        browser.get(url)
    except TimeoutException:
        print('get_page exception')
    
def get_page_nums():
    #总页面
    time.sleep(5)
    total = wait.until(EC.presence_of_all_elements_located((By.XPATH,'//*[@id="tbl_wrap"]/div[@class="pages"]')))
    global t
    if total[0].text:
        t = total[0].text[-4]
    else:
        t = 1
    return int(t)

def get_page_num():
    #当前页面
    global t
    if t != 1: 
        now = browser.find_element(By.XPATH,'//*[@id="tbl_wrap"]/div/span[@class="pagecurr pagedisabled"]')
        return now.text
    else:
        return '1'
    
def get_time():
    #当前时间
    nows = browser.find_element(By.XPATH,'//*[@id="bj_time"]')
    return nows.text

def next_page(page):
    print("正在爬取第"+get_page_num()+"页")
    try:
        if get_page_nums() ==1:
            html = browser.page_source
            parse_page(html)
            print("爬取完成")
        else:
            try:
                # 滑动到底部
                
                time.sleep(1)
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                # 一定要加time sleep不然网页加载不出来获取不到数据
                time.sleep(5)
                # 加载完成，获取当前页的网页资源
                wait.until(
                    EC.presence_of_all_elements_located(
                        (By.XPATH, '/html/body/div[3]/div[5]/div[3]/div[4]/div/a[4]')))
                html = browser.page_source
                # 解析当前页
                parse_page(html)
                #判断是否为最后一页
                if get_page_nums() == int(get_page_num()):
                    print("数据爬取完成")
                else:
                    # 开始翻页 获取到下一页的按钮
                    button = wait.until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, '#tbl_wrap > div > a:nth-last-child(1)')))
                    #tbl_wrap > div > span:nth-child(6)
                    button.click()

            except TimeoutException:
                print('next_page exception', page)
    except TimeoutException:
        print('next_page exception')
        
def parse_page(html):
    tree = etree.HTML(html)
    trs = tree.xpath('//*[@id="tbl_wrap"]/table/tbody/tr')
    
    all_lst=[]
    with alive_bar(len(trs)) as bar:
        for tr in trs:
            #判断是否可解析
            zhuti = tr.xpath('.//a//text()')
            try:
                code = zhuti[0] #代码
                name = zhuti[1] #名称
                stocks = tr.xpath('.//td//text()')
                price_new = stocks[0][1:] #最新价 
                count_range = stocks[1]#涨跌额
                extent_range = stocks[2] #涨跌幅
                buy = stocks[3] #买入
                sale = stocks[4] #卖出
                close_yesterday = stocks[5] #昨收
                open_today = stocks[6] #今开
                maxs = stocks[7] #最高
                mins = stocks[8] #最低
                amount_make = stocks[9] #成交量/手
                money_make = stocks[10] #成交额/万'''
                guba = tree.xpath('.//th[last()]/a/@href')[0]
            except:
                print('Parse failure')
            finally:
                # 汇总列表
                stockdict_values = [code,name,price_new,count_range,extent_range,
                         buy,sale,close_yesterday,open_today,
                         maxs,mins,amount_make,money_make,guba]
            #获得字典键
            stockdict_keys = tree.xpath('//*[@id="tbl_wrap"]/table/thead//text()')
            
            #生成字典
            stockdict = dict(zip(stockdict_keys,stockdict_values))
            
            all_lst.append(stockdict)
            
            bar()
            time.sleep(0.2)
            
        write_to_csv(all_lst)

def write_to_csv(all_lst):
    header = ['代码','名称','最新价','涨跌额','涨跌幅','买入','卖出','昨收','今开','最高','最低','成交量/手','成交额/万','股吧']
    with open('数据.csv', 'a', newline='',encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=header)  # 提前预览列名，当下面代码写入数据时，会将其一一对应。
        writer.writeheader()  # 写入列名
        writer.writerows(all_lst)  # 写入数据
        

In [6]:
if __name__ == '__main__':
    url = 'https://vip.stock.finance.sina.com.cn/mkt/#diyu_420000'
    options = webdriver.EdgeOptions()
    # 不加载图片
    options.add_experimental_option('prefs', {'profile.managed_default_content_settings.images': 2})
    browser = webdriver.Edge(options = options)
    wait = WebDriverWait(browser,60)
    get_page(url)
    print("爬取时间：",get_time())
    print("总页面：",get_page_nums())
    for page in range(1,get_page_nums()+1):
        next_page(page)

WebDriverException: Message: unknown error: cannot find msedge binary
Stacktrace:
Backtrace:
	Microsoft::Applications::Events::EventProperties::unpack [0x00F97AD3+60003]
	Microsoft::Applications::Events::ISemanticContext::SetTicket [0x00E8ECB1+712065]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00D1D3AE+4190]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00D394AE+39726]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00D36E49+29897]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00D631D8+211032]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00D629EE+209006]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00D5C7F6+183926]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00D3F5B9+64569]
	Microsoft::Applications::Events::GUID_t::GUID_t [0x00D402EF+67951]
	Microsoft::Applications::Events::ILogManager::DispatchEventBroadcast [0x011AB905+714101]
	Microsoft::Applications::Events::ILogManager::DispatchEventBroadcast [0x0111CFCC+130108]
	Microso

# 文本爬取

In [7]:
def req_data(url):
    try:
        httpheaders={'User-Agent':
                     'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}
        p=requests.get(url, headers=httpheaders)
        if p.status_code ==200:
            p.encoding='GBK'
            webhtml=p.text
            parse_data(webhtml)
            #return webhtml
        return None
    except RequestException:
        return None
    
def parse_data(webhtml):
    textlst1,textlst2=[],[]
    timelst1,timelst2=[],[]
    author1,author2=[],[]
    flag2,flag1=[],[] 
    remark1,remark2=[],[]
    whole=[]
    soup=BeautifulSoup(webhtml,'lxml')
    for date in soup.select('tr[class=""]'):
        remark1.append(date.find_all('td')[1].text)
        textlst1.append(date.find_all('td')[2].text)
        author1.append(date.find_all('td')[3].text)
        timelst1.append(date.find_all('td')[4].text)
        flag1.append(date.find_all('td')[0].text)
    x=soup.find_all('tr',class_='tr2')
    for item in x:
        remark2.append(item.find_all('td')[1].text)
        textlst2.append(item.find_all('td')[2].text)
        author2.append(date.find_all('td')[3].text)
        timelst2.append(item.find_all('td')[4].text)
        flag2.append(item.find_all('td')[0].text)
    #for author in soup.select('div[class="author"]'):
        #authorlst.append(author.text)
    textlst=list(itertools.chain.from_iterable(zip(textlst1,textlst2)))
    timelst=list(itertools.chain.from_iterable(zip(timelst1,timelst2)))
    author=list(itertools.chain.from_iterable(zip(author1,author2)))
    remark=list(itertools.chain.from_iterable(zip(remark1,remark2)))
    flag=list(itertools.chain.from_iterable(zip(flag1,flag2)))
    for i in [textlst,timelst,author,remark,flag]:
        whole.append(i)
    
    save_csv(whole)


        
def save_csv(self):
    data_dict = {'title':self[0],'time':self[1],'author':self[2],'flag':self[3],'remark':self[4]}
    data_dict_df = pd.DataFrame(data_dict)
    data_dict_df.to_csv('文本.csv',mode='a')
  
    

In [8]:
if __name__ == '__main__':
    urlbase=['https://guba.sina.com.cn/?s=bar&name=sz000783&type=0&page={}.html'.format(str(i+1))for i in range(14)]
    pool = Pool(17)
    pool.map(req_data,urlbase)
    print('文本爬取完成')

文本爬取完成
